## Some misc. code snippets 

In [1]:
%load_ext autoreload

In [9]:
%autoreload
# import libraries
import numpy as np
import pickle as pkl
import os
import sys
import torchvision.utils as vutils
import yaml

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import torch
%matplotlib inline

from celeba_dataset import CelebA
from unet_diffusion import UNet_Diffusion

In [10]:
#--------------------------------------------------------------------
# Read config
#--------------------------------------------------------------------
with open('./config/default.yaml', 'r') as file:
    try:
        config = yaml.safe_load(file)
    except yaml.YAMLError as exc:
        print(exc)

print(config)
# log_params(config)

diffusion_config = config['diffusion_params']
dataset_config = config['dataset_params']
model_config = config['model_params']
train_config = config['train_params']


{'dataset_params': {'train_path': '../data/img_align_celeba/img_align_celeba/train/', 'valid_path': '../data/img_align_celeba/img_align_celeba/valid/', 'limit_size': False, 'size_limit': -1}, 'diffusion_params': {'num_timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02, 'num_samples': 49, 'num_grid_rows': 7, 'sample_epochs': 2, 'task_name': 'default'}, 'model_params': {'img_size': [64, 64], 'time_emb_dim': 256, 'channels': [32, 64, 128, 256, 512], 'down_attn_type': ['linear', 'linear', 'linear', 'full'], 'down_channel_indices': [[0, 1], [1, 2], [2, 3], [3, 4]], 'mid_attn_type': ['full'], 'mid_channel_indices': [[4, 4]], 'up_attn_type': ['full', 'linear', 'linear', 'linear'], 'up_channel_indices': [[4, 3], [3, 2], [2, 1], [1, 0]], 'num_heads': 6, 'dim_head': 32, 'dropout': 0.0, 'attn_dropout': 0.0}, 'train_params': {'log_dir': './lightning_logs/', 'batch_size': 64, 'accumulate_grad_batches': 1, 'num_epochs': 500, 'checkpoint_name': 'None', 'log_every_nsteps': 1000, 'accelerator': '

In [11]:
from torchvision import transforms
from torchvision.transforms.v2 import Resize, Compose, ToDtype, RandomHorizontalFlip, RandomVerticalFlip 
from torchvision.transforms.v2 import RandomResizedCrop, RandomRotation, GaussianBlur, RandomErasing
from torch import utils

#--------------------------------------------------------------------
# Dataset, Dataloader
#--------------------------------------------------------------------
from pathlib import Path
image_dir_train = Path(dataset_config['train_path'])
image_dir_valid = Path(dataset_config['valid_path'])

img_size = tuple(model_config['img_size'])
batch_size = train_config['batch_size']

train_transforms = Compose([ToDtype(torch.float32, scale=False),
                            RandomHorizontalFlip(p=0.50),
                            Resize(img_size, antialias=True)
                            ])

# log_param('train_transforms', train_transforms)

train_dataset = CelebA(image_dir_train, 
                        transform=train_transforms, 
                        limit_size=dataset_config['limit_size'], 
                        size_limit=dataset_config['size_limit']) 

train_loader = utils.data.DataLoader(train_dataset, 
                                        batch_size=batch_size, 
                                        shuffle=True, 
                                        num_workers=5, 
                                        drop_last=True, 
                                        persistent_workers=True)


if image_dir_valid != 'None':
    validation_transforms = Compose([ToDtype(torch.float32, scale=False),
                                    Resize(img_size, antialias=True)
                                    ])
    # log_param('validation_transforms', validation_transforms)
    
    validation_dataset = CelebA(image_dir_valid, 
                                transform=validation_transforms, 
                                limit_size=False, 
                                size_limit=-1) 

    validation_loader = utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=5, persistent_workers=True)


Dataset has 182339 rows
Dataset has 20260 rows


In [13]:
%ls -l ../data/img_align_celeba/img_align_celeba/train/064850.jpg

-rw-rw-r-- 1 mark mark 8521 Oct  1  2019 ../data/img_align_celeba/img_align_celeba/train/064850.jpg


In [12]:
img1, img2 = train_dataset.__getitem__(0)
print()
print(img1.shape)

img_path: ../data/img_align_celeba/img_align_celeba/train/064850.jpg


AttributeError: '_OpNamespace' 'image' object has no attribute 'read_file'

In [6]:
data_iter = iter(train_loader)

batch = next(data_iter)
imgs = batch[0]
print(imgs.shape)

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/mark/anaconda3/envs/avm-dvm/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/mark/anaconda3/envs/avm-dvm/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mark/anaconda3/envs/avm-dvm/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/mark/dev/diffusion/celeba_dataset.py", line 33, in __getitem__
    image = read_image(img_path).to(torch.float32)
  File "/home/mark/anaconda3/envs/avm-dvm/lib/python3.9/site-packages/torchvision/io/image.py", line 258, in read_image
    data = read_file(path)
  File "/home/mark/anaconda3/envs/avm-dvm/lib/python3.9/site-packa

In [ ]:
%autoreload
import os
import torch
import glob
from torch import utils
from torch import nn
import pytorch_lightning as pl
from torchvision import transforms
from torchvision.transforms.v2 import Resize, Compose, ToDtype, RandomHorizontalFlip, RandomVerticalFlip 
from torchvision.transforms.v2 import RandomResizedCrop, RandomRotation, GaussianBlur, RandomErasing


#--------------------------------------------------------------------
# Dataset, Dataloader
#--------------------------------------------------------------------
from pathlib import Path
image_dir_train = Path('../data/img_align_celeba/img_align_celeba/train/')
image_dir_valid = Path('../data/img_align_celeba/img_align_celeba/valid/')

img_size = (64,64) 
batch_size = 8 


train_transforms = Compose([ToDtype(torch.float32, scale=False),
                            RandomHorizontalFlip(p=0.50),
                            # RandomVerticalFlip(p=0.25),
                            # transforms.RandomApply(nn.ModuleList([GaussianBlur(kernel_size=7)]), p=0.5),
                            # transforms.RandomApply(nn.ModuleList([RandomRotation(10.0)]), p=0.5),
                            # RandomResizedCrop(size=img_size, scale=(0.3, 1.0), antialias=True),
                            # RandomErasing(p=0.5, scale=(0.02, 0.20)),
                            Resize(img_size, antialias=True)
                            ])

valid_transforms = Compose([ToDtype(torch.float32, scale=False),
                            Resize(img_size, antialias=True)
                            ])

train_dataset = CelebA(image_dir_train, transform=train_transforms)
train_loader = utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True, num_workers=5, persistent_workers=True)

valid_dataset = CelebA(image_dir_valid, transform=valid_transforms)
valid_loader = utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle = False, num_workers=5, persistent_workers=True)


In [ ]:
# import glob
# image_dir_train = '../data/img_align_celeba/img_align_celeba/'
# img_list = glob.glob(image_dir_train+'*.jpg')
# print(len(img_list))

# num_train = int(len(img_list)*0.9)
# print('num_train:', num_train)
# num_valid = len(img_list) - num_train
# print('num valid:', num_valid)


In [ ]:
# import random
# # shuffle the list
# random.shuffle(img_list)
# train_file_names = img_list[0:num_train]
# valid_file_names = img_list[num_train:]

# print(len(train_file_names))
# print(len(valid_file_names))


In [ ]:

# print(train_file_names[0])
# print(os.path.abspath(train_file_names[0]))
# os.makedirs('../data/img_align_celeba/img_align_celeba/train/', exist_ok=True)
# os.makedirs('../data/img_align_celeba/img_align_celeba/valid/', exist_ok=True)

# # make 80/20 train/valid split


In [ ]:
# import shutil

# for i, f in enumerate(valid_file_names):
#     shutil.move(f, '../data/img_align_celeba/img_align_celeba/valid/')
#     if i % 100 == 0:
#         print(i)


In [ ]:
%ls -l '../data/img_align_celeba/img_align_celeba/train/' | wc -l
%ls -l '../data/img_align_celeba/img_align_celeba/valid/' | wc -l
%ls -l '../data/img_align_celeba/img_align_celeba/' | wc -l

In [ ]:
class UnNormalize(object):
    def __init__(self) : #, mean, std):
        pass
    def __call__(self, img):
        img = (img*127.5) + 127.5
        return img
    
unorm  = UnNormalize()

In [ ]:
imga, imgb = train_dataset.__getitem__(0)
print(imga.shape)

In [ ]:
import matplotlib.pyplot as plt

images, _  = next(iter(train_loader))
print(images.shape)
print(torch.min(images[0]), ', ', torch.max(images[0]))


cols = 4
rows = 4
print('num rows:', rows, ', num cols:', cols)
plt.figure(figsize=(10, 10))
idx = 0
for img in (images):  
    img = unorm(img).to(torch.uint8).permute(1, 2, 0)
    # target = unorm(target).to(torch.uint8).permute(1, 2, 0)

    idx += 1
    ax = plt.subplot(rows, cols, idx)
    ax.axis('off')
    plt.imshow(img)

    # idx += 1
    # ax = plt.subplot(rows, cols, idx)
    # ax.axis('off')
    # plt.imshow(target)

    if idx == (cols*rows):
        break

